In [ ]:
import torch
from torch_geometric.nn import GraphSAGE
import torch.nn as nn
from Stressnet import Modfiedunet4shrink
from ultility import SAGraphV3,cnntogra

# Graph_enhanced_CNN

In [ ]:
class Graph_enhanced_CNN_GraphSAGE(nn.Module):
    def __init__(self):
        super().__init__()
        self.c1=Modfiedunet4shrink(chin=1)
        self.g1=GraphSAGE(in_channels=int(8),hidden_channels=int(128), 
                                num_layers=4, out_channels=1)
        


    def forward(self, igra):
        x=igra.xdata128.float()
        first_channel = x[:, 0:1, :, :]
        x= first_channel

        gx,edge_index,posnew,pollinfor,batchinfo=igra.x,igra.edge_index,igra.pos.float(),igra.pollinfor,igra.batch
        gx=torch.cat((igra.x[:,0:2],igra.x[:,-3:],posnew),dim=-1).float()
        x1=self.c1(x)
        cnng=cnntogra(posnew,batchinfo,x1)
        # print(gx.shape,cnng.shape)
        gout=self.g1(torch.cat((gx,cnng),dim=-1),edge_index)
        return x1,gout


class Graph_enhanced_CNN_SA_GNN(nn.Module):
    def __init__(self,chlist=[8,32,64,128,128,128,128,128,64,32,9]):
        super().__init__()
        self.c1=Modfiedunet4shrink(chin=1)
        # self.g1=GraphSAGE(in_channels=int(8),hidden_channels=int(128), 
        #                         num_layers=4, out_channels=1)
        
        self.g1=SAGraphV3(in_channels=int(8),hidden_channels=int(128), 
                                num_layers=4, out_channels=1)

    def forward(self, igra):
        x=igra.xdata128.float()
        first_channel = x[:, 0:1, :, :]
        x= first_channel

        gx,edge_index,posnew,pollinfor,batchinfo=igra.x,igra.edge_index,igra.pos.float(),igra.pollinfor,igra.batch
        gx=torch.cat((igra.x[:,0:2],igra.x[:,-3:],posnew),dim=-1).float()
        x1=self.c1(x)
        cnng=cnntogra(posnew,batchinfo,x1)
        # print(gx.shape,cnng.shape)
        gout=self.g1(torch.cat((gx,cnng),dim=-1),edge_index)
        return x1,gout


# loss function

In [ ]:

def getgnnhighloss_mape(igra_svon, gx5final, igra_batch, percentile=80):

    # igra_svon shape [node_number, 1] # the stress value of the nodes
    # gx5final shape [node_number, 1] # the predicted stress value of the nodes
    # igra_batch shape [node_number] # the batch of the nodes to each design

    # 展平为一维数组，便于后续操作
    igra_svon = igra_svon.view(-1)
    gx5final = gx5final.view(-1)

    # 按照 batch 分组计算每个设计的百分位数
    unique_batches = torch.unique(igra_batch)
    percentiles = torch.zeros_like(unique_batches, dtype=torch.float32)

    # 计算每个设计的 80% 百分位数
    for i, batch in enumerate(unique_batches):
        mask = (igra_batch == batch)
        percentiles[i] = torch.quantile(igra_svon[mask], percentile / 100.0)

    # 将 percentile 映射回节点对应的批次
    batch_percentiles = percentiles[igra_batch]

    # 筛选出应力值大于百分位数阈值的节点
    high_stress_mask = (igra_svon >= batch_percentiles)

    # 计算筛选出的节点的 MAE
    real_high_stress = igra_svon[high_stress_mask]
    pred_high_stress = gx5final[high_stress_mask]

    # 确保存在超过阈值的节点，计算 MAE
    if real_high_stress.numel() > 0:
        avg_high_mae = torch.mean(torch.abs(real_high_stress - pred_high_stress)/real_high_stress)
    else:
        avg_high_mae = torch.tensor(0.0)

    return avg_high_mae


def getcnnhighloss_mape(Xtrain, predicted, Ytrain,percentile=80):
    """
    计算损失函数，包括整体损失和基于百分位数确定的高值区域损失。

    参数:
    Xtrain -- 输入数据其中第一个通道用于生成掩码标记Ytrain中的非负值
    predicted -- 预测值
    Ytrain -- 实际值
    percentile -- 用于确定高值区域的百分位数 (默认是80即高于80%的值)

    返回:
    total_loss -- 整体损失和高值区域损失的总和
    high_stress_loss -- 高值区域的损失
    """
    # 确保 Xtrain 第一个通道中值为1的位置, 这些是非负数的位置
    non_negative_mask = Ytrain[:, 0, :, :] != -10
    
    # 应用掩码，调整掩码的维度以匹配Ytrain和predicted的维度
    Ytrain_non_negative = Ytrain[:, 0, :, :][non_negative_mask]
    predicted_non_negative = predicted[:, 0, :, :][non_negative_mask]
    
    # 确定高值区域的阈值
    k = int(len(Ytrain_non_negative) * (1 - percentile / 100))
    threshold = Ytrain_non_negative.kthvalue(k).values if k > 0 else Ytrain_non_negative.min()
    
    # 高值掩码
    high_value_mask = Ytrain_non_negative >= threshold
    
    # 高值区域的数据
    Ytrain_high = Ytrain_non_negative[high_value_mask]
    predicted_high = predicted_non_negative[high_value_mask]
    
    # 高值区域的平均绝对误差
    high_stress_errors = torch.abs(predicted_high - Ytrain_high)/Ytrain_high
    high_stress_mean_error = torch.mean(high_stress_errors)
    
  
    
    return high_stress_mean_error


def getgnnhighloss(igra_svon, gx5final, igra_batch, percentile=80):

    # igra_svon shape [node_number, 1] # the stress value of the nodes
    # gx5final shape [node_number, 1] # the predicted stress value of the nodes
    # igra_batch shape [node_number] # the batch of the nodes to each design

    # 展平为一维数组，便于后续操作
    igra_svon = igra_svon.view(-1)
    gx5final = gx5final.view(-1)

    # 按照 batch 分组计算每个设计的百分位数
    unique_batches = torch.unique(igra_batch)
    percentiles = torch.zeros_like(unique_batches, dtype=torch.float32)

    # 计算每个设计的 80% 百分位数
    for i, batch in enumerate(unique_batches):
        mask = (igra_batch == batch)
        percentiles[i] = torch.quantile(igra_svon[mask], percentile / 100.0)

    # 将 percentile 映射回节点对应的批次
    batch_percentiles = percentiles[igra_batch]

    # 筛选出应力值大于百分位数阈值的节点
    high_stress_mask = (igra_svon >= batch_percentiles)

    # 计算筛选出的节点的 MAE
    real_high_stress = igra_svon[high_stress_mask]
    pred_high_stress = gx5final[high_stress_mask]

    # 确保存在超过阈值的节点，计算 MAE
    if real_high_stress.numel() > 0:
        avg_high_mae = torch.mean(torch.abs(real_high_stress - pred_high_stress))
    else:
        avg_high_mae = torch.tensor(0.0)

    return avg_high_mae



def getcnnhighloss(Xtrain, predicted, Ytrain,percentile=80):
    """
    计算损失函数，包括整体损失和基于百分位数确定的高值区域损失。

    参数:
    Xtrain -- 输入数据其中第一个通道用于生成掩码标记Ytrain中的非负值
    predicted -- 预测值
    Ytrain -- 实际值
    percentile -- 用于确定高值区域的百分位数 (默认是80即高于80%的值)

    返回:
    total_loss -- 整体损失和高值区域损失的总和
    high_stress_loss -- 高值区域的损失
    """
    # 确保 Xtrain 第一个通道中值为1的位置, 这些是非负数的位置
    non_negative_mask = Ytrain[:, 0, :, :] != -10
    
    # 应用掩码，调整掩码的维度以匹配Ytrain和predicted的维度
    Ytrain_non_negative = Ytrain[:, 0, :, :][non_negative_mask]
    predicted_non_negative = predicted[:, 0, :, :][non_negative_mask]
    
    # 确定高值区域的阈值
    k = int(len(Ytrain_non_negative) * (1 - percentile / 100))
    threshold = Ytrain_non_negative.kthvalue(k).values if k > 0 else Ytrain_non_negative.min()
    
    # 高值掩码
    high_value_mask = Ytrain_non_negative >= threshold
    
    # 高值区域的数据
    Ytrain_high = Ytrain_non_negative[high_value_mask]
    predicted_high = predicted_non_negative[high_value_mask]
    
    # 高值区域的平均绝对误差
    high_stress_errors = torch.abs(predicted_high - Ytrain_high)
    high_stress_mean_error = torch.mean(high_stress_errors)
    
  
    
    return high_stress_mean_error

